## Noisy reference tree construction

In [2]:
from NOISY_reference_tree_construction_functions import *
import mysql.connector as mariadb
from ete3 import Tree, TreeNode
import pandas as pd
import pandas as pd
import polars as pl
import numpy as np
import fileinput
import random
import time
import os
import re

## Reference tree construction for every group in a loop

In [3]:
#tax level and group name
#of group for main level (kingdom)

#define path to input and output
PATH_nodes = "/data/DBs/Taxonomy/13-05-2022/nodes.dmp"
PATH_speclist = "/data/DBs/Uniprot/speclist.txt"
PATH_TO_OUTPUT = "/data/ruslan_gumerov/reference_trees/"

#define lists for iteration
group_names_list = ["actinobacteria", "archaea", "proteobacteria", "fungi", "metazoa", "viridiplantae"]
parental_tax_list = ["phylum", "superkingdom", "phylum", "kingdom", "kingdom", "kingdom"]
parental_tax_id_list = ["201174", "2157", "1224", "4751", "33208", "33090"]
aggregation_tax_list = ["genus", "species", "genus", "family", "order", "genus"]

if __name__ == "__main__":
    for group, parental_tax_level, parental_tax_id, aggregation_tax_level in zip(group_names_list,
                                                                                parental_tax_list,
                                                                                parental_tax_id_list,
                                                                                aggregation_tax_list):
        df_nodes_with_parents = nodes_indexation(PATH_nodes)
        #get dataframe tax_id <-> mnemonics
        taxid_to_mnemonics_df = taxid_to_mnemonics_species(PATH_speclist, 59)
        #get dataframe tax_id <-> parent_tax_id <-> tax_level
        #for all species in parent_tax_level
        mariadb_connection = mariadb.connect(user="ruslan_gumerov",
                                         password="",
                                         database="swisspfam",
                                         unix_socket="/run/mysqld/mysqld.sock")
        swisspfam_mnemonics = get_table_names(mariadb_connection) 
        taxid_to_mnemonics_df.query("mnemonic in @swisspfam_mnemonics", inplace = True)
        taxid_to_mnemonics_df[parental_tax_level] = taxid_to_mnemonics_df.index.to_series().apply(find_tax_level_for_tax_id,
                                                                              input_df_taxonomy = df_nodes_with_parents,
                                                                              desired_level = parental_tax_level)
        taxid_to_mnemonics_df = taxid_to_mnemonics_df[taxid_to_mnemonics_df[parental_tax_level] == parental_tax_id]
        taxid_to_mnemonics_df[aggregation_tax_level] = taxid_to_mnemonics_df.index.to_series().apply(find_tax_level_for_tax_id,
                                                                              input_df_taxonomy = df_nodes_with_parents,
                                                                              desired_level = aggregation_tax_level)
        taxid_to_mnemonics_df.dropna(inplace=True)

        tree_df = tree_dataframe_construction(df_nodes_with_parents, taxid_to_mnemonics_df[aggregation_tax_level])
        tree_df.insert(0, -1, tree_df.index)
        tree_df.columns = [x for x in range(0,tree_df.shape[1])]
        with open(f"{PATH_TO_OUTPUT}{group}_{aggregation_tax_level}_tree.nwk", "w") as outf:
            newick_tree = ete3_tree_from_df(tree_df)
            print(newick_tree)
            #outf.write(newick_tree)
        print(f"{PATH_TO_OUTPUT}{group}_{aggregation_tax_level}_tree.nwk")
        mariadb_connection.close()

             0      1     2       3        4  5       6  7
tax_id                                                    
1269      1268  85006  1760  201174  1783272  2  131567  1
1573     85023  85006  1760  201174  1783272  2  131567  1
1645      1268  85006  1760  201174  1783272  2  131567  1
1663      1268  85006  1760  201174  1783272  2  131567  1
1678     31953  85004  1760  201174  1783272  2  131567  1
...        ...    ...   ...     ...      ... ..     ... ..
1742992   1268  85006  1760  201174  1783272  2  131567  1
1742993   1268  85006  1760  201174  1783272  2  131567  1
1866885   1762  85007  1760  201174  1783272  2  131567  1
1912215  31957  85009  1760  201174  1783272  2  131567  1
1912216  31957  85009  1760  201174  1783272  2  131567  1

[84 rows x 8 columns]
((((((((1269,1645,1663,32207,57493,1742989,1742992,1742993),(1573,33882,110932),(1696),(1707),(2038),(13242),(43668),(43673),(53357),(60919),(84756),(157920,186188,254250)),((1678,2701,196081,196082)),((1716),(

              0     1       2        3        4       5       6        7   \
tax_id                                                                      
6049     1779146  6042    6040    33208    33154    2759  131567        1   
6103        6102  6101    6073     6072    33208   33154    2759   131567   
6125        6102  6101    6073     6072    33208   33154    2759   131567   
6180        6179  6178    6157  1206795  2697495   33317   33213     6072   
6193        6179  6178    6157  1206795  2697495   33317   33213     6072   
...          ...   ...     ...      ...      ...     ...     ...      ...   
2507557    30302  7712    7711    33511    33213    6072   33208    33154   
2558200     8825  8782  436492   436491   436489  436486    8492  1329799   
2682553  2682552  7735    7711    33511    33213    6072   33208    33154   
2785018     7674  7638    7625     7624   133551    7586   33511    33213   
2899670  2899756  6676    6675   116172  2172821    6657  197562   197563   